## Alumno: Rafael Navarro Gómez

# Práctica: Interacción con Amazon S3 mediante la librería boto3

Esta práctica tiene como objetivo introducirte al uso de la librería **boto3**, el SDK oficial de Amazon Web Services (AWS) para Python, utilizada para interactuar con servicios en la nube como S3.

💡 *Boto3 es totalmente compatible con AWS S3 y puede utilizarse también con otros servicios compatibles con el protocolo S3, como MinIO.*

En esta práctica trabajaremos con una instancia **local de MinIO**, configurada para comportarse igual que un servicio S3 real.

**Objetivos:**
- Conocer la estructura básica de la API de boto3.
- Practicar las operaciones esenciales sobre buckets y objetos (crear, listar, subir, descargar, eliminar).
- Desarrollar autonomía en la gestión de almacenamiento distribuido S3 desde Python.


💡 *Para eliminiar todos el contenido previo que pueda existir en S3, se recomienda ejecutar el código de borrado del apartado 9 de esta actividad.*

## 1. Configuración inicial del entorno

Antes de utilizar boto3, debemos asegurarnos de que la librería esté instalada y configurada con las credenciales de acceso al servicio MinIO local. Estas credenciales son equivalentes a las de AWS S3 y pueden definirse mediante variables de entorno.

In [1]:
import os

# Configuración de credenciales (MinIO local)
os.environ['AWS_ACCESS_KEY_ID'] = 'minioadmin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minioadmin'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

import boto3

# Creación del cliente de S3 apuntando al endpoint de MinIO local
s3 = boto3.client('s3', endpoint_url='http://localhost:9000')
print('Cliente S3 configurado correctamente')

Cliente S3 configurado correctamente


## 2. Listado de buckets disponibles

Con el método `list_buckets()` podemos listar todos los buckets disponibles en el servidor S3 (MinIO).

In [2]:
response = s3.list_buckets()
print('Buckets disponibles:')
for bucket in response['Buckets']:
    print(f" - {bucket['Name']}")

Buckets disponibles:
 - bucket-rafa-navarro
 - nuevo-bucket-practica
 - nuevo-directorio-sync


## 3. Creación de un nuevo bucket

El método `create_bucket()` permite crear un nuevo bucket donde almacenaremos nuestros archivos (objetos).

In [3]:
bucket_name = 'mi-bucket-practica-python'
try:
    s3.create_bucket(Bucket=bucket_name)
    print(f'Bucket "{bucket_name}" creado correctamente.')
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f'El bucket "{bucket_name}" ya existe.')

Bucket "mi-bucket-practica-python" creado correctamente.


## 4. Subida de archivos al bucket

Para subir archivos desde el sistema local al bucket, utilizamos el método `upload_file()`.

In [4]:
# Creamos un archivo local de prueba
with open('ejemplo-python.txt', 'w') as f:
    f.write('Ejemplo de archivo de prueba para S3 con boto3')

# Subida del archivo
s3.upload_file('ejemplo-python.txt', bucket_name, 'ejemplo-python.txt')
print('Archivo subido correctamente al bucket.')

Archivo subido correctamente al bucket.


## 5. Listar el contenido de un bucket

Podemos obtener el listado de objetos almacenados en un bucket utilizando el método `list_objects_v2()`.

In [19]:
response = s3.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    print('Objetos en el bucket:')
    for obj in response['Contents']:
        print(f" - {obj['Key']} ({obj['Size']} bytes)")
else:
    print('El bucket está vacío.')

Objetos en el bucket:
 - archivo_1.txt (23 bytes)
 - archivo_2.txt (23 bytes)
 - rafa.txt (46 bytes)


## 6. Descarga de archivos desde el bucket

Para descargar un archivo del bucket, usamos el método `download_file()`.

In [6]:
s3.download_file(bucket_name, 'ejemplo-python.txt', 'ejemplo-python_descargado.txt')
print('Archivo descargado correctamente.')

# Verificamos el contenido
with open('ejemplo-python_descargado.txt', 'r') as f:
    print('Contenido del archivo descargado:')
    print(f.read())

Archivo descargado correctamente.
Contenido del archivo descargado:
Ejemplo de archivo de prueba para S3 con boto3


## 7. Eliminación de objetos y buckets

Para eliminar un objeto, utilizamos `delete_object()`. Los buckets pueden eliminarse con `delete_bucket()` (siempre que estén vacíos).

In [7]:
# Eliminamos un objeto
s3.delete_object(Bucket=bucket_name, Key='ejemplo-python.txt')
print('Objeto eliminado correctamente.')

# Eliminamos el bucket
try:
    s3.delete_bucket(Bucket=bucket_name)
    print('Bucket eliminado correctamente.')
except Exception as e:
    print('Error al eliminar el bucket:', e)

Objeto eliminado correctamente.
Bucket eliminado correctamente.


## 8. Sincronización de múltiples archivos (subida en lote)

Podemos automatizar la subida de varios archivos utilizando un pequeño bucle en Python.

In [8]:
import os
os.makedirs('python_datos_locales', exist_ok=True)
for i in range(3):
    with open(f'datos_locales/archivo_{i}.txt', 'w') as f:
        f.write(f'Contenido del archivo {i}')

bucket_sync = 'mi-bucket-sync'
s3.create_bucket(Bucket=bucket_sync)

for archivo in os.listdir('python_datos_locales'):
    s3.upload_file(f'datos_locales/{archivo}', bucket_sync, archivo)
print('Archivos sincronizados correctamente.')

Archivos sincronizados correctamente.


### 9 Eliminación de todos los buckets del sistema

En este ejemplo aprenderás a eliminar todos los *buckets* existentes en el sistema S3, sin importar si contienen objetos o versiones.  
Esta operación es especialmente útil en entornos de desarrollo o pruebas, cuando se desea limpiar por completo el espacio de trabajo.

> ⚠️ **Advertencia:** Esta acción eliminará *todos los buckets y sus contenidos de forma permanente*.  
> Utilízala únicamente en entornos controlados, como un despliegue local con **MinIO**.

La función utiliza la librería `boto3`, totalmente compatible con **Amazon S3** y con servidores **MinIO** cuando se especifica el parámetro `endpoint_url`.  
El proceso consiste en tres pasos:
1. Recorrer todos los buckets disponibles.  
2. Eliminar todos los objetos (y versiones) que contengan.  
3. Borrar finalmente el bucket vacío.


In [9]:
import boto3

def eliminar_todos_los_buckets():
    """
    Elimina todos los buckets del sistema S3, incluyendo su contenido.
    Compatible con AWS S3 y MinIO.
    """
    # Configuración del cliente S3
    s3 = boto3.resource(
        's3',
        endpoint_url='http://localhost:9000',  # Cambia o elimina esta línea si usas AWS
        aws_access_key_id='minioadmin',
        aws_secret_access_key='minioadmin',
        region_name='us-east-1'
    )

    # Iterar sobre todos los buckets existentes
    for bucket in s3.buckets.all():
        print(f"Eliminando bucket: {bucket.name}")

        # Eliminar todos los objetos del bucket
        bucket.objects.all().delete()
        # Eliminar también las versiones si existen
        bucket.object_versions.all().delete()

        # Eliminar el bucket vacío
        bucket.delete()

    print("✅ Todos los buckets han sido eliminados correctamente.")

# Ejemplo de uso:
eliminar_todos_los_buckets()


Eliminando bucket: bucket-rafa-navarro
Eliminando bucket: mi-bucket-sync
Eliminando bucket: nuevo-bucket-practica
Eliminando bucket: nuevo-directorio-sync
✅ Todos los buckets han sido eliminados correctamente.


# 10. Actividades prácticas para el alumnado

A continuación se proponen ejercicios que consolidan los conocimientos adquiridos.
Realiza cada uno en la celda de código correspondiente.

### Ejercicio 1
Crea un bucket con tu nombre utilizando boto3.

In [10]:
bucket_name = 'rafa-navarro'
try:
    s3.create_bucket(Bucket=bucket_name)
    print(f'Bucket "{bucket_name}" creado correctamente.')
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f'El bucket "{bucket_name}" ya existe.')

Bucket "rafa-navarro" creado correctamente.


### Ejercicio 2
Crea un archivo de texto y súbelo a tu bucket.

In [11]:
# Creamos un archivo local de prueba
with open('rafa.txt', 'w') as f:
    f.write('Ejemplo de archivo de prueba para S3 con boto3')

# Subida del archivo
s3.upload_file('rafa.txt', bucket_name, 'rafa.txt')
print('Archivo subido correctamente al bucket.')

Archivo subido correctamente al bucket.


### Ejercicio 3
Lista los objetos de tu bucket y muestra su tamaño.

In [16]:
response = s3.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    print('Objetos en el bucket:')
    for obj in response['Contents']:
        print(f" - {obj['Key']} ({obj['Size']} bytes)")
else:
    print('El bucket está vacío.')

Objetos en el bucket:
 - archivo_0.txt (23 bytes)
 - archivo_1.txt (23 bytes)
 - archivo_2.txt (23 bytes)
 - rafa.txt (46 bytes)


### Ejercicio 4
Descarga un archivo y muestra su contenido por pantalla.

In [13]:
s3.download_file(bucket_name, 'rafa.txt', 'rafa_descargado.txt')
print('Archivo descargado correctamente.')

# Verificamos el contenido
with open('rafa.txt', 'r') as f:
    print('Contenido del archivo descargado:')
    print(f.read())

Archivo descargado correctamente.
Contenido del archivo descargado:
Ejemplo de archivo de prueba para S3 con boto3


### Ejercicio 5
Crea varios archivos locales y súbelos al bucket en un bucle.

In [15]:
import os
os.makedirs('s3_python', exist_ok=True)
for i in range(3):
    with open(f's3_python/archivo_{i}.txt', 'w') as f:
        f.write(f'Contenido del archivo {i}')

bucket_sync = 'rafa-navarro'

for archivo in os.listdir('s3_python'):
    s3.upload_file(f's3_python/{archivo}', bucket_sync, archivo)
print('Archivos sincronizados correctamente.')

Archivos sincronizados correctamente.


### Ejercicio 6
Elimina un archivo del bucket y verifica que ha desaparecido.

In [21]:
# Eliminamos un objeto
s3.delete_object(Bucket=bucket_name, Key='archivo_0.txt')
print('Objeto eliminado correctamente.')

NoSuchBucket: An error occurred (NoSuchBucket) when calling the DeleteObject operation: The specified bucket does not exist

### Ejercicio 7
Elimina el bucket (asegúrate primero de vaciarlo).

In [20]:
response = s3.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    for obj in response['Contents']:
        s3.delete_object(Bucket=bucket_name, Key=obj['Key'])
        print('Objeto eliminado correctamente.')
else:
    print('El bucket está vacío.')


# Eliminamos el bucket
try:
    s3.delete_bucket(Bucket=bucket_name)
    print('Bucket eliminado correctamente.')
except Exception as e:
    print('Error al eliminar el bucket:', e)

Objeto eliminado correctamente.
Objeto eliminado correctamente.
Objeto eliminado correctamente.
Bucket eliminado correctamente.


### Ejercicio 8
Crea un script que suba todos los archivos de un directorio a un bucket especificado por el usuario.

In [23]:
bucket_name = 'specific-bucket'
try:
    s3.create_bucket(Bucket=bucket_name)
    print(f'Bucket "{bucket_name}" creado correctamente.')
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f'El bucket "{bucket_name}" ya existe.')

for archivo in os.listdir('s3_python'):
    s3.upload_file(f's3_python/{archivo}', bucket_name, archivo)
print('Archivos sincronizados correctamente.')

El bucket "specific-bucket" ya existe.
Archivos sincronizados correctamente.


### Ejercicio 9
Descarga todo el contenido de un bucket a una carpeta local nueva.

In [28]:
# import os
# os.mkdir("new_locale")

response = s3.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    for obj in response['Contents']:
        local_path = os.path.join('new_locale', os.path.basename(obj['Key']))
        s3.download_file(bucket_name, obj['Key'], local_path)
        print('Archivo descargado correctamente.')
else:
    print('El bucket está vacío.')

Archivo descargado correctamente.
Archivo descargado correctamente.
Archivo descargado correctamente.


### Ejercicio 10
Modifica el script anterior para que solo descargue los archivos con extensión `.txt`.

In [ ]:
# import os
# os.mkdir("new_locale")

response = s3.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    for obj in response['Contents']:
        if obj['Key'].endswith(.txt):
            local_path = os.path.join('new_locale', os.path.basename(obj['Key']))
            s3.download_file(bucket_name, obj['Key'], local_path)
            print('Archivo descargado correctamente.')
else:
    print('El bucket está vacío.')

---
## Antes de terminar
Guarda el notebook y expórtalo para entregarlo a través de la plataforma del curso.